# 爬取巨牛汇所有人才信息

In [1]:
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import requests
import random
import time
import threading

In [2]:
class MyThread(threading.Thread):

    def __init__(self, func, args=()):
        super(MyThread, self).__init__()
        self.func = func
        self.args = args

    def run(self):
        self.result = self.func(*self.args)

    def get_result(self):
        try:
            return self.result
        except Exception as e:
            return None

In [3]:
class SpiderCow(object):

    def __init__(self, start_url, main_url, pages):
        self.main_url = main_url
        self.start_url = start_url
        self.pages = pages
        self.url_tail = []

    def _make_url(self):
        for i in range(1, self.pages + 1):
            self.url_tail.append("{}?randomKey={}".format(i, random.randint(1000, 9999)))

    def _request_url(self, url):
        headers = {'User-Agent': UserAgent(path="/home/ubuntu/svn_project/mytest/ju_niu_hui/user_agent.json").random}
        res = None
        try:
            res = requests.get(url=url, headers=headers)
        except Exception as e:
            print(f"请求异常, 异常url: {url}, 异常信息: {e}")
        return res

    def run(self):
        thread_list = []
        if not self.url_tail:
            self._make_url()
            
        for i in self.url_tail:
            t = MyThread(self._request_url, args=(self.start_url + i, ))
            thread_list.append(t)
            
        for th in thread_list:
            th.start()

        for th in thread_list:
            th.join()
            res = th.get_result()
            self._analysis_html(res)


    def _analysis_html(self, response):
        res_html = response.text
        res_soup = BeautifulSoup(res_html, "lxml")
        user_list = res_soup.find_all(name='div', attrs={"class": "list"})
        for one_usr in user_list[0]:
            try:
                user_url = self.main_url + one_usr.a["href"]    # 用户链接
                user_name = one_usr.h3.next_element             # 用户名称
                user_position = one_usr.h3.contents[2].text     # 用户岗位
                user_team = one_usr.h3.contents[3].text         # 个人或团队
                user_address = one_usr.h3.contents[4].text      # 用户地址
                user_seniority = one_usr.h3.contents[5].text    # 用户工龄
                user_skills = one_usr.p.text                    # 用户技能
                try:
                    user_order = one_usr.contents[1].span.text      # 用户订单
                except:
                    user_order = "完成 0 单"
                try:
                    user_charges = one_usr.contents[1].em.text      # 用户收费
                except:
                    user_charges = "0元/日"
                user_product = one_usr.p.next_sibling.text      # 用户作品
                print(f"用户名称: {user_name}\t用户岗位: {user_position}\t用户地址: {user_address}", end="")
                print(f"\t个人或团队: {user_team}\t用户工龄: {user_seniority}\t用户订单: {user_order}", end="")
                print(f"\t收费标准: {user_charges}")
                print(f"用户技能: {user_skills}")
                print(f"用户作品: {user_product}")
                print(f"用户链接: {user_url}")
                print("-" * 100)
            except Exception as e:
                pass

    def get_totle_pages(self):  # 获取总的页数
        res = self._request_url(self.start_url)
        res_html = res.text
        res_soup = BeautifulSoup(res_html, "lxml")
        total_pages = res_soup.find_all(name='span', attrs={"class": "num"})[0].contents[-1].text
        print(total_pages)

    def _get_user_introduce(self, user_url):
        res = self._request_url(url=user_url)
        res_html = res.text
        res_soup = BeautifulSoup(res_html, "lxml")
        usr_list = res_soup.find_all(name='p', attrs={"class": "introduce"})
        return usr_list[0].text

In [7]:
main_url = "https://www.juniuhui.com"
start_url = "https://www.juniuhui.com/talentPool/talentList/index/"
pages = 102

In [5]:
spider_cow = SpiderCow(start_url=start_url, main_url=main_url, pages=pages)

### 获取总页数

In [6]:
spider_cow.get_totle_pages()

102


In [8]:
spider_cow.run()

用户名称: passing	用户岗位: APP开发工程师	用户地址: 郑州市	个人或团队: 团队	用户工龄: 4年	用户订单: 完成 0 单	收费标准: 0元/日
用户技能: 技能：APP设计，APP产品，后台产品，产品咨询，iOS，Java，Android
用户作品: 作品：大学时代社区，畅聊
用户链接: https://www.juniuhui.com/personalCenter/newResume/20259.html
----------------------------------------------------------------------------------------------------
用户名称: 软件开发后生仔丶	用户岗位: 技术总监	用户地址: 临沂市	个人或团队: 个人	用户工龄: 8年	用户订单: 完成 0 单	收费标准: 500元/日
用户技能: 技能：HTML5，PHP，小程序，iOS，Android，Linux，数据库，C++，网站设计，原型设计，网站产品，APP产品，后台产品，需求撰写，产品咨询
用户作品: 作品：小马直播
用户链接: https://www.juniuhui.com/personalCenter/newResume/21103.html
----------------------------------------------------------------------------------------------------
用户名称: 杭州天泽	用户岗位: 技术负责人	用户地址: 杭州市	个人或团队: 团队	用户工龄: 10年及以上	用户订单: 完成 0 单	收费标准: 0元/日
用户技能: 技能：APP产品，数据库，Java，小程序，C++，Python
用户作品: 作品：海通证券-互联网金融综合理财服务平台，海通证券网上营业厅，北美百应
用户链接: https://www.juniuhui.com/personalCenter/newResume/17538.html
----------------------------------------------------------------------------------------------------
用户名称: